In [2]:
heart_lst = [
    "normal_heart_sound","split_first_heart_sound","split_second_heart_sound","third_heart_sound","fourth_heart_sound",
    "functional_murmur_sound","diastolic_murmur_sound","opening_snap_sound","holosystolic_murmur_sound","early_systolic_murmur_sound","mid_systolic_murmur_sound","continuous_murmur_sound","austin_flint_murmur_sound","pericardial_rub_sound","graham_steell_murmur_sound",
    "aortic_valve_regurgitation_sound","aortic_valve_stenosis_sound","aortic_valve_stenosis_regurgitation_sound","congenital_aortic_stenosis_sound",
    "mitral_valve_regurgitation_sound","mitral_valve_stenosis_sound","mitral_valve_prelapse_sound","mitral_stenosis_regurgitation_sound","mitral_stenosis_tricuspid_regurgitation_sound",
    "pulmonary_valve_stenosis_sound","pulmonary_valve_regurgitation_sound",
    "tricuspid_valve_regurgitation_sound",
    "coarctation_of_the_aorta_sound","hypertrophic_cardiomyopathy_sound","patent_ductus_arteriosus_sound","atrial_septal_defect_sound","ventricular_septal_defect_sound","acute_myocardial_infarction_sound","congestive_heart_failure_sound","systemic_hypertension_sound","acute_pericarditis_sound","dilated_cardiomyopathy_sound","pulmonary_hypertension_sound","tetralogy_of_fallot_sound","ventricular_aneurysm_sound","ebstein_anomaly_sound"
]

In [3]:
valve_name = [
    'mitral_valve',
    'aortic_valve',
    'pulmonary_valve',
    'tricuspid_valve',
    'erbs_point'
]

In [4]:
mitral_ranges = []
aortic_ranges = []
pulmonary_ranges = []
tricuspid_ranges = []
erb_ranges = []

In [5]:
for x in heart_lst:
    mitral_ranges.append(x + "_" + valve_name[0] + "_range")
    aortic_ranges.append(x + "_" + valve_name[1] + "_range")
    pulmonary_ranges.append(x + "_" + valve_name[2] + "_range")
    tricuspid_ranges.append(x + "_" + valve_name[3] + "_range")
    erb_ranges.append(x + "_" + valve_name[4] + "_range")

In [7]:
for x in mitral_ranges:
    print("\"{}\"".format(x),end=",")

"normal_heart_sound_mitral_valve_range","split_first_heart_sound_mitral_valve_range","split_second_heart_sound_mitral_valve_range","third_heart_sound_mitral_valve_range","fourth_heart_sound_mitral_valve_range","functional_murmur_sound_mitral_valve_range","diastolic_murmur_sound_mitral_valve_range","opening_snap_sound_mitral_valve_range","holosystolic_murmur_sound_mitral_valve_range","early_systolic_murmur_sound_mitral_valve_range","mid_systolic_murmur_sound_mitral_valve_range","continuous_murmur_sound_mitral_valve_range","austin_flint_murmur_sound_mitral_valve_range","pericardial_rub_sound_mitral_valve_range","graham_steell_murmur_sound_mitral_valve_range","aortic_valve_regurgitation_sound_mitral_valve_range","aortic_valve_stenosis_sound_mitral_valve_range","aortic_valve_stenosis_regurgitation_sound_mitral_valve_range","congenital_aortic_stenosis_sound_mitral_valve_range","mitral_valve_regurgitation_sound_mitral_valve_range","mitral_valve_stenosis_sound_mitral_valve_range","mitral_valv

In [8]:
for x in aortic_ranges:
    print("\"{}\"".format(x),end=",")

"normal_heart_sound_aortic_valve_range","split_first_heart_sound_aortic_valve_range","split_second_heart_sound_aortic_valve_range","third_heart_sound_aortic_valve_range","fourth_heart_sound_aortic_valve_range","functional_murmur_sound_aortic_valve_range","diastolic_murmur_sound_aortic_valve_range","opening_snap_sound_aortic_valve_range","holosystolic_murmur_sound_aortic_valve_range","early_systolic_murmur_sound_aortic_valve_range","mid_systolic_murmur_sound_aortic_valve_range","continuous_murmur_sound_aortic_valve_range","austin_flint_murmur_sound_aortic_valve_range","pericardial_rub_sound_aortic_valve_range","graham_steell_murmur_sound_aortic_valve_range","aortic_valve_regurgitation_sound_aortic_valve_range","aortic_valve_stenosis_sound_aortic_valve_range","aortic_valve_stenosis_regurgitation_sound_aortic_valve_range","congenital_aortic_stenosis_sound_aortic_valve_range","mitral_valve_regurgitation_sound_aortic_valve_range","mitral_valve_stenosis_sound_aortic_valve_range","mitral_valv

In [9]:
for x in pulmonary_ranges:
    print("\"{}\"".format(x),end=",")

"normal_heart_sound_pulmonary_valve_range","split_first_heart_sound_pulmonary_valve_range","split_second_heart_sound_pulmonary_valve_range","third_heart_sound_pulmonary_valve_range","fourth_heart_sound_pulmonary_valve_range","functional_murmur_sound_pulmonary_valve_range","diastolic_murmur_sound_pulmonary_valve_range","opening_snap_sound_pulmonary_valve_range","holosystolic_murmur_sound_pulmonary_valve_range","early_systolic_murmur_sound_pulmonary_valve_range","mid_systolic_murmur_sound_pulmonary_valve_range","continuous_murmur_sound_pulmonary_valve_range","austin_flint_murmur_sound_pulmonary_valve_range","pericardial_rub_sound_pulmonary_valve_range","graham_steell_murmur_sound_pulmonary_valve_range","aortic_valve_regurgitation_sound_pulmonary_valve_range","aortic_valve_stenosis_sound_pulmonary_valve_range","aortic_valve_stenosis_regurgitation_sound_pulmonary_valve_range","congenital_aortic_stenosis_sound_pulmonary_valve_range","mitral_valve_regurgitation_sound_pulmonary_valve_range","

In [11]:
for x in tricuspid_ranges:
    print("\"{}\"".format(x),end=",")

"normal_heart_sound_tricuspid_valve_range","split_first_heart_sound_tricuspid_valve_range","split_second_heart_sound_tricuspid_valve_range","third_heart_sound_tricuspid_valve_range","fourth_heart_sound_tricuspid_valve_range","functional_murmur_sound_tricuspid_valve_range","diastolic_murmur_sound_tricuspid_valve_range","opening_snap_sound_tricuspid_valve_range","holosystolic_murmur_sound_tricuspid_valve_range","early_systolic_murmur_sound_tricuspid_valve_range","mid_systolic_murmur_sound_tricuspid_valve_range","continuous_murmur_sound_tricuspid_valve_range","austin_flint_murmur_sound_tricuspid_valve_range","pericardial_rub_sound_tricuspid_valve_range","graham_steell_murmur_sound_tricuspid_valve_range","aortic_valve_regurgitation_sound_tricuspid_valve_range","aortic_valve_stenosis_sound_tricuspid_valve_range","aortic_valve_stenosis_regurgitation_sound_tricuspid_valve_range","congenital_aortic_stenosis_sound_tricuspid_valve_range","mitral_valve_regurgitation_sound_tricuspid_valve_range","

In [12]:
for x in erb_ranges:
    print("\"{}\"".format(x),end=",")

"normal_heart_sound_erbs_point_range","split_first_heart_sound_erbs_point_range","split_second_heart_sound_erbs_point_range","third_heart_sound_erbs_point_range","fourth_heart_sound_erbs_point_range","functional_murmur_sound_erbs_point_range","diastolic_murmur_sound_erbs_point_range","opening_snap_sound_erbs_point_range","holosystolic_murmur_sound_erbs_point_range","early_systolic_murmur_sound_erbs_point_range","mid_systolic_murmur_sound_erbs_point_range","continuous_murmur_sound_erbs_point_range","austin_flint_murmur_sound_erbs_point_range","pericardial_rub_sound_erbs_point_range","graham_steell_murmur_sound_erbs_point_range","aortic_valve_regurgitation_sound_erbs_point_range","aortic_valve_stenosis_sound_erbs_point_range","aortic_valve_stenosis_regurgitation_sound_erbs_point_range","congenital_aortic_stenosis_sound_erbs_point_range","mitral_valve_regurgitation_sound_erbs_point_range","mitral_valve_stenosis_sound_erbs_point_range","mitral_valve_prelapse_sound_erbs_point_range","mitral